In [86]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.max_columns = 100

In [87]:
byCategory=pd.read_csv('byCategory.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [88]:
byCategory.loc[byCategory.index==220816,'quantity_multiplier']=180

In [89]:
byCategory.loc[byCategory.index==220352,'quantity_multiplier']=48

In [90]:
byCategory[byCategory.index==220352]

,GUID,seller_GUID,seller_name,subtotal_guid,is_returned,create_time,business_hour_guid,order_from_type,user_id,installment,mobile_pay_type,business_hour_order_time_s,business_hour_order_time_e,business_hour_deliver_time_s,business_hour_deliver_time_e,pic_alt,zip_code,city_name,district_name,item_name,item_orig_price,item_price,item_default_daily_amount,event_name,event_title,product_spec,shopping_cart,combo_pack_count,quantity_multiplier,city_list,deal_acc_business_group_id,acc_business_group_name,deal_type,is_travel_deal,installment_3months,installment_6months,installment_12months,deny_installment,shippingdate,is_long_contract,develope_sales_id,operation_sales_id,slug,ordered_quantity,ordered_total,category_list,freight_amount,enable_isp,discount_price,is_wms,ancestor_business_hour_guid,code_name,code,code_second,product_launch_month,product_launch_hour,product_launch_weekday,campagin_period,order_deliver_days,is_Presales,product_launch_year,list,unit_price,is_Longtern,category_unit_price,compare_cate_unit_price,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,len_list,is_Discount,total_sales,sales_3days,ratio_3days,Unnamed: 0,hottimes,total_sales_amount,sales_7days,ratio_7days,sales_14days,ratio_14days,sales_30days,ratio_30days,sales_60days,ratio_60days,sales_180days,ratio_180days
220352,44363241-44C3-45C5-88FD-783843083A9D,5AE9C827-0AE6-4029-8168-BED6171BF86A,凱創實業股份有限公司,1225.0,0,2019-02-07 06:15:22.143,4B9FEA30-6B13-4A59-85EB-734BEE8A4DD3,2,1113079961,0,NaN,2019-01-29 00:00:00.000,2019-12-31 23:59:00.000,2019-01-29 00:00:00.000,2020-01-01 23:59:00.000,【3M】細滑牙線棒散裝促銷包/牙線/3M/細滑牙線/牙線棒/3m牙線,116,台北市,文山區,【3M】細滑牙線棒散裝促銷包(附贈隨身盒)(4包/組)X12PK/箱 1箱/組,1990.0,1225.0,20,只要1|225元(含運)即可享有原價1|990元【3M】細滑牙線棒散裝促銷包(附贈隨身盒)(...,17Life破盤超低價回饋◆雙線設計加倍潔淨 ◆有效清除齒縫殘渣 ◆線材強韌不易斷裂 ◆不易...,【3M】細滑牙線棒散裝促銷包(36支x3包+40支x1包)&nbsp;<br />◆型號：D...,1,1,48.0,[477],7,宅配,2112,0,1,1,1,0,0.0,0,1112453309,1.112453e+09,NaN,27,33075.0,[88|100176|100177|100249|100254|100401],0.0,0,NaN,1,NaN,口罩,513.0,2112,1,0,1,336,0,0,2019,['4B9FEA30-6B13-4A59-85EB-734BEE8A4DD3'],1225.0,1,4.995,245.245,1.0,1225.0,1.0,1225.0,1,1,18.0,0.0,0.0,527688,0,22050.0,0.0,0.0,2450.0,0.111,4900.0,0.222,19600.0,0.889,22050.0,1.0


In [91]:
## 刪除捐錢公益活動
byCategory=byCategory.drop(index=98836)
byCategory=byCategory.reset_index(drop=True)

In [92]:
##將檔期時間調整為至少一個月(檔期開始時間最晚為2019-03-30)
byCategory=byCategory[byCategory.business_hour_order_time_s<'2019-03-30']

In [93]:
##找出檔期長度超過3年,檔期長度即設為1100天
byCategory.loc[byCategory.campagin_period>1100,'campagin_period']=1100

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x1a1916f860>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [94]:
byCategory['unit_price']=byCategory['item_price']/byCategory['quantity_multiplier']


In [120]:
byCategory.columns

Index(['GUID', 'seller_GUID', 'seller_name', 'subtotal_guid', 'is_returned',
       'create_time', 'business_hour_guid', 'order_from_type', 'user_id',
       'installment', 'mobile_pay_type', 'business_hour_order_time_s',
       'business_hour_order_time_e', 'business_hour_deliver_time_s',
       'business_hour_deliver_time_e', 'pic_alt', 'zip_code', 'city_name',
       'district_name', 'item_name', 'item_orig_price', 'item_price',
       'item_default_daily_amount', 'event_name', 'event_title',
       'product_spec', 'shopping_cart', 'combo_pack_count',
       'quantity_multiplier', 'city_list', 'deal_acc_business_group_id',
       'acc_business_group_name', 'deal_type', 'is_travel_deal',
       'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'shippingdate', 'is_long_contract',
       'develope_sales_id', 'operation_sales_id', 'slug', 'ordered_quantity',
       'ordered_total', 'category_list', 'freight_amount', 'enable_isp',
       'di

In [121]:
df=byCategory[['business_hour_guid', 'subtotal_guid', 'item_orig_price', 'item_price',
       'quantity_multiplier', 'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'shippingdate', 'freight_amount', 'enable_isp',
       'is_Presales', 'is_Longtern', 
       'code', 'product_launch_month', 'product_launch_hour',
       'product_launch_weekday', 'campagin_period', 'order_deliver_days',
       'product_launch_year', 'unit_price',
       'category_unit_price', 'compare_cate_unit_price',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'is_Discount', 'total_sales', 'hottimes',
       'total_sales_amount', 'sales_7days', 'sales_14days',
       'sales_30days', 'sales_60days','sales_180days','ratio_7days',
       'ratio_14days', 'ratio_30days', 
       'ratio_60days', 'ratio_180days']]

In [122]:
df.shape

(223696, 42)

In [123]:
#發現 code,category_unit_price,compare_cate_unit_price 有nan值 因為分類關係
df.isna().sum()

business_hour_guid          0
subtotal_guid               0
item_orig_price             0
item_price                  0
quantity_multiplier         0
installment_3months         0
installment_6months         0
installment_12months        0
deny_installment            0
shippingdate                0
freight_amount              0
enable_isp                  0
is_Presales                 0
is_Longtern                 0
code                       26
product_launch_month        0
product_launch_hour         0
product_launch_weekday      0
campagin_period             0
order_deliver_days          0
product_launch_year         0
unit_price                  0
category_unit_price        26
compare_cate_unit_price    26
quantity_sold_guid          0
subtotal_allguid            0
quantity_sold_allguid       0
avg_unit_price              0
is_Discount                 0
total_sales                 0
hottimes                    0
total_sales_amount          0
sales_7days                 0
sales_14da

In [124]:
#把沒有分類的 歸為同一類
df['code']=df['code'].fillna(0)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x1a1918dba8>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [125]:
df['category_unit_price']=df['category_unit_price'].fillna(df['unit_price'])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
df['compare_cate_unit_price']=df['compare_cate_unit_price'].fillna(1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 新增欄位price_gap : item_orig_price - item_price   售價與定價之價格差

In [127]:
df['price_gap']=df['item_orig_price']-df['item_price']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [128]:
df.head(3)

,business_hour_guid,subtotal_guid,item_orig_price,item_price,quantity_multiplier,installment_3months,installment_6months,installment_12months,deny_installment,shippingdate,freight_amount,enable_isp,is_Presales,is_Longtern,code,product_launch_month,product_launch_hour,product_launch_weekday,campagin_period,order_deliver_days,product_launch_year,unit_price,category_unit_price,compare_cate_unit_price,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,is_Discount,total_sales,hottimes,total_sales_amount,sales_7days,sales_14days,sales_30days,sales_60days,sales_180days,ratio_7days,ratio_14days,ratio_30days,ratio_60days,ratio_180days,price_gap
0,732103DD-F831-4210-8449-6B1917AA37F5,699.0,1350.0,699.0,90.0,0,0,0,0,0.0,0.0,0,0,0,413.0,1,12,3,21,-1,2015,7.766667,145.0,0.054,90.0,699.0,90.0,7.766667,1,3870.0,0,30057.0,13281.0,18174.0,30057.0,30057.0,30057.0,0.442,0.605,1.0,1.0,1.0,651.0
1,E4849B75-FC79-4B2E-AF00-DF82E59B2872,594.0,1794.0,594.0,6.0,0,0,0,0,0.0,0.0,0,0,0,166.0,1,12,3,13,-1,2015,99.000000,359.0,0.276,6.0,594.0,6.0,99.000000,1,102.0,0,10098.0,7722.0,10098.0,10098.0,10098.0,10098.0,0.765,1.000,1.0,1.0,1.0,1200.0
2,61B117B3-9E4C-4809-9992-0F89E740E5E9,469.0,1800.0,469.0,1.0,0,0,0,0,0.0,0.0,0,0,0,1604.0,1,12,3,1,-1,2015,469.000000,319.0,1.470,1.0,469.0,1.0,469.000000,1,216.0,0,101304.0,101304.0,101304.0,101304.0,101304.0,101304.0,1.000,1.000,1.0,1.0,1.0,1331.0


## 將檔案順序打散

In [129]:
df=df.sample(frac=1, replace=False, random_state=1)

In [130]:
df=df.reset_index(drop=True)

將資料分為輸入變數與目標變數

## 特徵欄位進行標準化

In [131]:
corrtable=(df.corr()[df.corr().index=='sales_7days']).T
sales_7days=corrtable.sort_values(ascending=False,by='sales_7days')
sales_7days=sales_7days.drop(index=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days'])
sales_7days=sales_7days.reset_index()
sales_7days.columns=['features','sales_7days']
sales_7days.head(3)

,features,sales_7days
0,total_sales_amount,0.602282
1,subtotal_guid,0.153186
2,subtotal_allguid,0.137619


In [132]:
corrtable=(df.corr()[df.corr().index=='sales_14days']).T
sales_14days=corrtable.sort_values(ascending=False,by='sales_14days')
sales_14days=sales_14days.drop(index=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days'])
sales_14days=sales_14days.reset_index()
sales_14days.columns=['features','sales_14days']
sales_14days.head(3)

,features,sales_14days
0,total_sales_amount,0.717857
1,subtotal_guid,0.151698
2,subtotal_allguid,0.136509


In [133]:
corrtable=(df.corr()[df.corr().index=='sales_30days']).T
sales_30days=corrtable.sort_values(ascending=False,by='sales_30days')
sales_30days=sales_30days.drop(index=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days'])
sales_30days=sales_30days.reset_index()
sales_30days.columns=['features','sales_30days']
sales_30days.head(3)

,features,sales_30days
0,total_sales_amount,0.824244
1,subtotal_guid,0.162766
2,subtotal_allguid,0.145378


In [134]:
corrtable=(df.corr()[df.corr().index=='sales_60days']).T
sales_60days=corrtable.sort_values(ascending=False,by='sales_60days')
sales_60days=sales_60days.drop(index=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days'])
sales_60days=sales_60days.reset_index()
sales_60days.columns=['features','sales_60days']
sales_60days.head(3)

,features,sales_60days
0,total_sales_amount,0.901688
1,subtotal_guid,0.165720
2,subtotal_allguid,0.148162


In [135]:
corrtable=(df.corr()[df.corr().index=='sales_180days']).T
sales_180days=corrtable.sort_values(ascending=False,by='sales_180days')
sales_180days=sales_180days.drop(index=['sales_7days','sales_14days','sales_30days','sales_60days','sales_180days'])
sales_180days=sales_180days.reset_index()
sales_180days.columns=['features','sales_180days']
sales_180days.head(3)

,features,sales_180days
0,total_sales_amount,0.970415
1,subtotal_guid,0.163950
2,item_price,0.149084


In [26]:
table=pd.merge(sales_7days,sales_14days)
table=pd.merge(table,sales_30days)
table=pd.merge(table,sales_60days)
table=pd.merge(table,sales_180days)
table.head()

,features,sales_7days,sales_14days,sales_30days,sales_60days,sales_180days
0,total_sales_amount,0.602282,0.717857,0.824244,0.901688,0.970415
1,subtotal_guid,0.153186,0.151698,0.162766,0.165720,0.163950
2,subtotal_allguid,0.137619,0.136509,0.145378,0.148162,0.147190
3,item_price,0.119662,0.125943,0.136672,0.145663,0.149084
4,item_orig_price,0.096709,0.104815,0.112598,0.117592,0.121350


In [27]:
table

,features,sales_7days,sales_14days,sales_30days,sales_60days,sales_180days
0,total_sales_amount,0.602282,0.717857,0.824244,0.901688,0.970415
1,subtotal_guid,0.153186,0.151698,0.162766,0.165720,0.163950
2,subtotal_allguid,0.137619,0.136509,0.145378,0.148162,0.147190
3,item_price,0.119662,0.125943,0.136672,0.145663,0.149084
4,item_orig_price,0.096709,0.104815,0.112598,0.117592,0.121350
5,avg_unit_price,0.096142,0.106289,0.117968,0.126020,0.129725
6,unit_price,0.096031,0.106128,0.117693,0.125776,0.129323
7,total_sales,0.080430,0.093980,0.109550,0.115893,0.133064
8,code,0.059193,0.063271,0.068698,0.076381,0.074615
9,category_unit_price,0.056837,0.063450,0.073307,0.084541,0.097408


In [ ]:
table.to_csv('corr_table.csv',index=False)

In [ ]:
## 類別型特徵:
###       ['code', 'zip_code', 'product_launch_month', 'product_launch_hour','product_launch_weekday', 'enable_isp', 'is_returned', 'is_Presales','is_Longtern', 'installment_3months', 'installment_6months','installment_12months', 'deny_installment', 'is_Discount']   
## 數值型特徵:
### [ 'subtotal_guid', 'item_orig_price', 'item_price', 'quantity_multiplier','shippingdate', 'freight_amount', 'campagin_period','product_launch_year', 'unit_price', 'category_unit_price','compare_cate_unit_price', 'quantity_sold_guid', 'subtotal_allguid',
###       'quantity_sold_allguid', 'avg_unit_price', 'sales_3days', 'hottimes','25%_days', '50%_days', '75%_days', 'price_gap', 'sales_7days','sales_14days', 'sales_30days', 'sales_60days', 'sales_180days']


## 因為code的類別數較大,故使用計數編碼

In [136]:
code=df.code.value_counts().to_frame()
code=code.rename(columns={'code':'code_counts'})
code.index.name='code'
#code.columns=[['code','code_counts']]
code=code.reset_index(drop=False)
code.head(3)

,code,code_counts
0,536.0,28490
1,423.0,23094
2,1604.0,18231


In [137]:
df=pd.merge(df,code)
df.head(3)

,business_hour_guid,subtotal_guid,item_orig_price,item_price,quantity_multiplier,installment_3months,installment_6months,installment_12months,deny_installment,shippingdate,freight_amount,enable_isp,is_Presales,is_Longtern,code,product_launch_month,product_launch_hour,product_launch_weekday,campagin_period,order_deliver_days,product_launch_year,unit_price,category_unit_price,compare_cate_unit_price,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,is_Discount,total_sales,hottimes,total_sales_amount,sales_7days,sales_14days,sales_30days,sales_60days,sales_180days,ratio_7days,ratio_14days,ratio_30days,ratio_60days,ratio_180days,price_gap,code_counts
0,F26E2C4C-EF71-4605-B2FE-37985DD695AE,2399.0,3280.0,2399.0,1.0,1,1,1,0,7.0,0.0,0,0,0,187.0,8,6,4,3,-1,2018,2399.0,740.0,3.242,1.0,2399.0,1.0,2399.0,1,20.0,0,47980.0,47980.0,47980.0,47980.0,47980.0,47980.0,1.0,1.0,1.0,1.00,1.0,881.0,290
1,E567BC2A-3FBA-4F87-85BC-A517E4458E27,599.0,2364.0,599.0,1.0,1,1,1,0,0.0,0.0,0,0,1,187.0,12,15,3,365,-1,2018,599.0,740.0,0.809,1.0,599.0,1.0,599.0,1,4.0,0,2396.0,1198.0,1198.0,1198.0,1797.0,2396.0,0.5,0.5,0.5,0.75,1.0,1765.0,290
2,F7944384-61F6-44BA-9631-1E09FB19DCD5,738.0,1800.0,738.0,2.0,1,1,1,0,0.0,0.0,0,0,1,187.0,12,15,3,365,-1,2018,369.0,740.0,0.499,2.0,738.0,2.0,369.0,1,2.0,0,738.0,0.0,0.0,0.0,738.0,738.0,0.0,0.0,0.0,1.00,1.0,1062.0,290


In [138]:
df=df.drop(columns=['code'])
df.head()

,business_hour_guid,subtotal_guid,item_orig_price,item_price,quantity_multiplier,installment_3months,installment_6months,installment_12months,deny_installment,shippingdate,freight_amount,enable_isp,is_Presales,is_Longtern,product_launch_month,product_launch_hour,product_launch_weekday,campagin_period,order_deliver_days,product_launch_year,unit_price,category_unit_price,compare_cate_unit_price,quantity_sold_guid,subtotal_allguid,quantity_sold_allguid,avg_unit_price,is_Discount,total_sales,hottimes,total_sales_amount,sales_7days,sales_14days,sales_30days,sales_60days,sales_180days,ratio_7days,ratio_14days,ratio_30days,ratio_60days,ratio_180days,price_gap,code_counts
0,F26E2C4C-EF71-4605-B2FE-37985DD695AE,2399.0,3280.0,2399.0,1.0,1,1,1,0,7.0,0.0,0,0,0,8,6,4,3,-1,2018,2399.0,740.0,3.242,1.0,2399.0,1.0,2399.0,1,20.0,0,47980.0,47980.0,47980.0,47980.0,47980.0,47980.0,1.0,1.00,1.0,1.00,1.0,881.0,290
1,E567BC2A-3FBA-4F87-85BC-A517E4458E27,599.0,2364.0,599.0,1.0,1,1,1,0,0.0,0.0,0,0,1,12,15,3,365,-1,2018,599.0,740.0,0.809,1.0,599.0,1.0,599.0,1,4.0,0,2396.0,1198.0,1198.0,1198.0,1797.0,2396.0,0.5,0.50,0.5,0.75,1.0,1765.0,290
2,F7944384-61F6-44BA-9631-1E09FB19DCD5,738.0,1800.0,738.0,2.0,1,1,1,0,0.0,0.0,0,0,1,12,15,3,365,-1,2018,369.0,740.0,0.499,2.0,738.0,2.0,369.0,1,2.0,0,738.0,0.0,0.0,0.0,738.0,738.0,0.0,0.00,0.0,1.00,1.0,1062.0,290
3,8FF9E8A6-A847-4FEF-9A25-DC3883C57180,558.0,2560.0,558.0,2.0,1,1,1,0,0.0,0.0,1,0,1,2,15,2,365,-1,2019,279.0,740.0,0.377,2.0,558.0,2.0,279.0,1,8.0,0,2232.0,0.0,558.0,2232.0,2232.0,2232.0,0.0,0.25,1.0,1.00,1.0,2002.0,290
4,D3D32534-AAE6-4AE5-9C89-336C7B1163CF,389.0,599.0,389.0,2.0,1,1,1,0,0.0,0.0,0,0,1,11,16,4,365,-1,2018,194.5,740.0,0.263,2.0,389.0,2.0,194.5,1,4.0,0,778.0,389.0,389.0,389.0,389.0,778.0,0.5,0.50,0.5,0.50,1.0,210.0,290


In [139]:
df['orig_unit_price']=df['item_orig_price']/df['quantity_multiplier']

In [140]:
df.columns

Index(['business_hour_guid', 'subtotal_guid', 'item_orig_price', 'item_price',
       'quantity_multiplier', 'installment_3months', 'installment_6months',
       'installment_12months', 'deny_installment', 'shippingdate',
       'freight_amount', 'enable_isp', 'is_Presales', 'is_Longtern',
       'product_launch_month', 'product_launch_hour', 'product_launch_weekday',
       'campagin_period', 'order_deliver_days', 'product_launch_year',
       'unit_price', 'category_unit_price', 'compare_cate_unit_price',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'is_Discount', 'total_sales', 'hottimes',
       'total_sales_amount', 'sales_7days', 'sales_14days', 'sales_30days',
       'sales_60days', 'sales_180days', 'ratio_7days', 'ratio_14days',
       'ratio_30days', 'ratio_60days', 'ratio_180days', 'price_gap',
       'code_counts', 'orig_unit_price'],
      dtype='object')

In [111]:
dflist=df.columns
nwlist=df_new.columns
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

diff(dflist,nwlist)


['business_hour_guid',
 'subtotal_guid',
 'item_orig_price',
 'item_price',
 'shippingdate',
 'freight_amount',
 'order_deliver_days',
 'product_launch_year',
 'quantity_sold_guid',
 'subtotal_allguid',
 'quantity_sold_allguid',
 'total_sales',
 'total_sales_amount']

In [141]:

##舊商品新檔次
df_temp=df[['code_counts','subtotal_guid', 'item_orig_price', 'item_price',
       'quantity_multiplier', 'installment_3months', 'installment_6months',
       'installment_12months', 'deny_installment', 'shippingdate',
       'freight_amount', 'enable_isp', 'is_Presales', 'is_Longtern',
       'product_launch_month', 'product_launch_hour', 'product_launch_weekday',
       'campagin_period', 'order_deliver_days', 'product_launch_year',
       'unit_price', 'category_unit_price', 'compare_cate_unit_price',
       'quantity_sold_guid', 'subtotal_allguid', 'quantity_sold_allguid',
       'avg_unit_price', 'is_Discount',  'hottimes',
       'sales_7days', 'sales_14days', 'sales_30days',
       'sales_60days', 'sales_180days', 'price_gap', 
       'orig_unit_price', 'total_sales_amount','ratio_7days', 'ratio_14days',
       'ratio_30days', 'ratio_60days', 'ratio_180days']]
all_ohe=pd.get_dummies(df_temp,columns=['product_launch_month','product_launch_hour','product_launch_weekday'])
ohe_x=all_ohe.drop(columns=[ 'subtotal_guid','order_deliver_days','quantity_sold_guid','sales_7days', 'sales_14days', 'sales_30days','sales_60days', 'sales_180days','ratio_7days', 'ratio_14days',
       'ratio_30days', 'ratio_60days', 'ratio_180days'])
#ohe_y=ohe_df[['sales_7days', 'sales_14days', 'sales_30days','sales_60days', 'sales_180days']]
ohe_y=all_ohe[['sales_7days', 'total_sales_amount','ratio_7days']]



In [60]:
#新商品新檔次
df_new=df[['product_launch_month', 'product_launch_hour', 'product_launch_weekday',
       'enable_isp', 'is_Presales', 'is_Longtern', 'code_counts',
       'installment_3months', 'installment_6months', 'installment_12months',
       'deny_installment', 'is_Discount', 'quantity_multiplier', 'orig_unit_price',
       'campagin_period','unit_price','category_unit_price', 'compare_cate_unit_price', 
       'avg_unit_price','hottimes', 'price_gap', 'sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days']]
ohe_df=pd.get_dummies(df_new,columns=['product_launch_month','product_launch_hour','product_launch_weekday'])
ohe_x=ohe_df.drop(columns=['sales_7days', 'sales_14days', 'sales_30days','sales_60days', 'sales_180days'])
#ohe_y=ohe_df[['sales_7days', 'sales_14days', 'sales_30days','sales_60days', 'sales_180days']]
ohe_y=ohe_df[['sales_7days']]


In [33]:
##標準化
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ohe_df_scaled = StandardScaler().fit_transform(ohe_df)


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [142]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ohe_x, ohe_y, test_size=0.1, random_state=2)

In [ ]:
## 舊商品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
#model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

In [67]:
## 新產品新檔期
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(X_train,y_train)
Y_pred=model.predict(X_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test,Y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, Y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,Y_pred))

mean squared error: 156464611.97
mean_absolute_error: 3131.48
r2_score: 0.13


In [39]:
def trainTestValSplit(df):
    # 將資料切分為訓練(80%),測試(10%)和驗證(10%)三份。
    train=df.sample(frac=0.8)
    test=df.drop( train.index )
    val=test.sample(frac=0.5)
    test=test.drop( val.index)
    return train,test,val

def dfXYSplit(df,targetName):
    # 將特徵和目標變數切成兩份資料。
    
    dfX=df.drop(columns=targetName)
    dfY=df[targetName]
    
    return dfX,dfY
train,test,val=trainTestValSplit(ohe_df)

trainX,trainY=dfXYSplit(train,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])
testX,testY=dfXYSplit(test,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])
valX,valY=dfXYSplit(val,['sales_7days', 'sales_14days','sales_30days', 'sales_60days', 'sales_180days'])

In [40]:
##先預測前7天之銷量
trainY=trainY['sales_7days']
testY=testY['sales_7days']
valY=valY['sales_7days']

## DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=0)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

## RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(max_depth=2, random_state=0,n_estimators=100)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

In [432]:
valY.head(10)

110818     838.0
186979    4245.0
132928       0.0
207490       0.0
200408     390.0
121335     960.0
212677       0.0
122059    1533.0
92399      489.0
107586       0.0
Name: sales_7days, dtype: float64

In [433]:
valPredY[:10]

array([ 838., 4245.,    0.,    0.,  390.,  960.,    0., 1533.,  489.,
          0.])

### XGBoost

In [52]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

mean squared error: 115719356.85
mean_absolute_error: 3029.00
r2_score: 0.11


### SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
model=SVR(gamma='scale', C=1.0, epsilon=0.2)
model.fit(trainX,trainY)
testPredY=model.predict(testX)
valPredY=model.predict(valX)
print("mean squared error: %.2f"
      % mean_squared_error(valY, valPredY))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(valY, valPredY))
print("r2_score: %.2f"
      % r2_score(valY,valPredY))

In [ ]:
ohe_df

# 僅將資料分為測試集與驗證集 進行建模

In [50]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(ohe_x, ohe_y, test_size=0.1, random_state=4)

In [53]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
#model = MultiOutputRegressor(xgb.XGBRegressor(n_estimators=200)).fit(x_train, y_train)
model = xgb.XGBRegressor(n_estimators=100)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
print("r2_score: %.2f"
      % r2_score(y_test,y_pred))

mean squared error: 272841798.05
mean_absolute_error: 3375.75
r2_score: 0.09


## 使用網格搜尋法

In [ ]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
clf= xgb.XGBRegressor()
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
grid_search = GridSearchCV(clf, param_grid,n_jobs=-1, verbose=1)
grid_result = grid_search.fit(x_train, y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
grid_result.best_params_
clf_bestparam = xgb.XGBRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)
print("mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
print("r2_score: %.2f"
      % r2_score(y_test, y_pred))


In [ ]:
##若
def neg(x):
    if x <0 :
        return 0
    else:
        return x
y_test['predict']=y_pred
y_test['predict']=y_test['predict'].apply(neg)
pred=y_test['predict']
test=y_test['sales_7days']
print("mean squared error: %.2f"
      % mean_squared_error(test, pred))

print("mean_absolute_error: %.2f"
      % mean_absolute_error(test, pred))
print("r2_score: %.2f"
      % r2_score(test, pred))